In [1]:
# Machine Learning Data Preprocessing Lab - Week 3
# Topics: Missing Data Handling, Feature Scaling, Encoding, Binning, Normalization, Standardization

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, load_wine
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Create sample dataset with missing values
def create_sample_data_with_missing():
    """Create sample dataset with intentional missing values"""
    data = {
        'age': [25, 30, np.nan, 35, 40, np.nan, 45, 50],
        'salary': [50000, 60000, 55000, np.nan, 70000, 65000, np.nan, 80000],
        'experience': [2, 5, 3, np.nan, 8, 6, 10, 12],
        'department': ['IT', 'HR', 'IT', 'Finance', np.nan, 'IT', 'HR', 'Finance'],
        'performance_score': [8.5, 7.2, np.nan, 9.1, 8.8, 7.5, np.nan, 9.3]
    }
    return pd.DataFrame(data)

# Create dataset
df_missing = create_sample_data_with_missing()

In [3]:
df_missing

,age,salary,experience,department,performance_score
0,25.0,50000.0,2.0,IT,8.5
1,30.0,60000.0,5.0,HR,7.2
2,NaN,55000.0,3.0,IT,NaN
3,35.0,NaN,NaN,Finance,9.1
4,40.0,70000.0,8.0,NaN,8.8
5,NaN,65000.0,6.0,IT,7.5
6,45.0,NaN,10.0,HR,NaN
7,50.0,80000.0,12.0,Finance,9.3


In [4]:
# Create dataset
df_missing = create_sample_data_with_missing()
print("Original dataset with missing values:")
print(df_missing)
print(f"\nMissing values count:\n{df_missing.isnull().sum()}")

Original dataset with missing values:
    age   salary  experience department  performance_score
0  25.0  50000.0         2.0         IT                8.5
1  30.0  60000.0         5.0         HR                7.2
2   NaN  55000.0         3.0         IT                NaN
3  35.0      NaN         NaN    Finance                9.1
4  40.0  70000.0         8.0        NaN                8.8
5   NaN  65000.0         6.0         IT                7.5
6  45.0      NaN        10.0         HR                NaN
7  50.0  80000.0        12.0    Finance                9.3

Missing values count:
age                  2
salary               2
experience           1
department           1
performance_score    2
dtype: int64


In [5]:
# Method 1: Drop rows with missing values
df_drop_rows = df_missing.dropna()
print(f"\nAfter dropping rows with missing values: {df_drop_rows.shape}")


After dropping rows with missing values: (3, 5)


In [6]:
df_drop_rows

,age,salary,experience,department,performance_score
0,25.0,50000.0,2.0,IT,8.5
1,30.0,60000.0,5.0,HR,7.2
7,50.0,80000.0,12.0,Finance,9.3


In [7]:
# Method 2: Drop columns with missing values
df_drop_cols = df_missing.dropna(axis=1)
df_drop_cols

""
0
1
2
3
4
5
6
7


In [8]:
# Method 3: Fill missing values with mean/mode
df_fill_mean = df_missing.copy()
# Fill numerical columns with mean
numerical_cols = ['age', 'salary', 'experience', 'performance_score']
for col in numerical_cols:
    df_fill_mean[col].fillna(df_fill_mean[col].mean(), inplace=True)

In [9]:
df_fill_mean

,age,salary,experience,department,performance_score
0,25.0,50000.000000,2.000000,IT,8.5
1,30.0,60000.000000,5.000000,HR,7.2
2,37.5,55000.000000,3.000000,IT,8.4
3,35.0,63333.333333,6.571429,Finance,9.1
4,40.0,70000.000000,8.000000,NaN,8.8
5,37.5,65000.000000,6.000000,IT,7.5
6,45.0,63333.333333,10.000000,HR,8.4
7,50.0,80000.000000,12.000000,Finance,9.3


In [10]:
# Fill categorical columns with mode
df_fill_mean['department'].fillna(df_fill_mean['department'].mode()[0], inplace=True)
print(f"\nAfter filling with mean/mode:")
df_fill_mean


After filling with mean/mode:


,age,salary,experience,department,performance_score
0,25.0,50000.000000,2.000000,IT,8.5
1,30.0,60000.000000,5.000000,HR,7.2
2,37.5,55000.000000,3.000000,IT,8.4
3,35.0,63333.333333,6.571429,Finance,9.1
4,40.0,70000.000000,8.000000,IT,8.8
5,37.5,65000.000000,6.000000,IT,7.5
6,45.0,63333.333333,10.000000,HR,8.4
7,50.0,80000.000000,12.000000,Finance,9.3


In [11]:
# Method 4: Forward fill and backward fill
df_ffill = df_missing.fillna(method='ffill')
df_bfill = df_missing.fillna(method='bfill')
df_ffill

,age,salary,experience,department,performance_score
0,25.0,50000.0,2.0,IT,8.5
1,30.0,60000.0,5.0,HR,7.2
2,30.0,55000.0,3.0,IT,7.2
3,35.0,55000.0,3.0,Finance,9.1
4,40.0,70000.0,8.0,Finance,8.8
5,40.0,65000.0,6.0,IT,7.5
6,45.0,65000.0,10.0,HR,7.5
7,50.0,80000.0,12.0,Finance,9.3


In [12]:
df_bfill

,age,salary,experience,department,performance_score
0,25.0,50000.0,2.0,IT,8.5
1,30.0,60000.0,5.0,HR,7.2
2,35.0,55000.0,3.0,IT,9.1
3,35.0,70000.0,8.0,Finance,9.1
4,40.0,70000.0,8.0,IT,8.8
5,45.0,65000.0,6.0,IT,7.5
6,45.0,80000.0,10.0,HR,9.3
7,50.0,80000.0,12.0,Finance,9.3


In [13]:
# Method 5: Using SimpleImputer
imputer_mean = SimpleImputer(strategy='mean')
imputer_mode = SimpleImputer(strategy='most_frequent')

df_imputed = df_missing.copy()
df_imputed[numerical_cols] = imputer_mean.fit_transform(df_imputed[numerical_cols])
df_imputed[['department']] = imputer_mode.fit_transform(df_imputed[['department']])
print(f"\nAfter SimpleImputer:")
df_imputed


After SimpleImputer:


,age,salary,experience,department,performance_score
0,25.0,50000.000000,2.000000,IT,8.5
1,30.0,60000.000000,5.000000,HR,7.2
2,37.5,55000.000000,3.000000,IT,8.4
3,35.0,63333.333333,6.571429,Finance,9.1
4,40.0,70000.000000,8.000000,IT,8.8
5,37.5,65000.000000,6.000000,IT,7.5
6,45.0,63333.333333,10.000000,HR,8.4
7,50.0,80000.000000,12.000000,Finance,9.3


In [14]:
# Method 6: KNN Imputer (for numerical data)
knn_imputer = KNNImputer(n_neighbors=2)
df_knn = df_missing.copy()
df_knn[numerical_cols] = knn_imputer.fit_transform(df_knn[numerical_cols])
print(f"\nAfter KNN Imputer (numerical columns only):")
print(df_knn[numerical_cols])


After KNN Imputer (numerical columns only):
    age   salary  experience  performance_score
0  25.0  50000.0         2.0               8.50
1  30.0  60000.0         5.0               7.20
2  35.0  55000.0         3.0               7.85
3  35.0  67500.0         7.0               9.10
4  40.0  70000.0         8.0               8.80
5  40.0  65000.0         6.0               7.50
6  45.0  75000.0        10.0               9.05
7  50.0  80000.0        12.0               9.30


In [15]:
# Load sample dataset (Boston Housing - using alternative since it's deprecated)
# Creating synthetic housing data
np.random.seed(42)
housing_data = pd.DataFrame({
    'rooms': np.random.normal(6, 1.5, 100),
    'age': np.random.uniform(1, 100, 100),
    'distance': np.random.exponential(3, 100),
    'tax_rate': np.random.uniform(200, 800, 100),
    'price': np.random.normal(25, 10, 100)
})

print("Original housing dataset (first 5 rows):")
print(housing_data.head())
print(f"\nDataset statistics:")
print(housing_data.describe())

Original housing dataset (first 5 rows):
      rooms        age  distance    tax_rate      price
0  6.745071  42.323689  4.751476  413.583607  29.129315
1  5.792604  22.988673  2.095306  654.707666  19.362754
2  6.971533  12.866671  2.580468  208.636093  16.777796
3  8.284545  34.423902  2.034880  269.643584  27.436872
4  5.648770  94.348061  0.651645  227.601585  27.449666

Dataset statistics:
            rooms         age    distance    tax_rate       price
count  100.000000  100.000000  100.000000  100.000000  100.000000
mean     5.844230   49.134412    3.186273  496.588464   25.796529
std      1.362253   28.526248    2.882322  180.942077   10.017056
min      2.070382    1.501097    0.046732  206.502591    3.761043
25%      5.098641   24.886689    1.035238  356.203319   18.708445
50%      5.809566   51.231218    2.368628  495.810854   25.708066
75%      6.608928   69.772938    4.494577  653.221210   32.130967
max      8.778417   98.579395   13.831709  794.303085   46.898029


In [16]:
housing_data.sample(5)

,rooms,age,distance,tax_rate,price
37,3.060495,19.465333,3.636863,743.210385,17.326524
82,8.216841,65.544165,0.077033,770.364288,45.607479
36,6.313295,32.757226,3.582223,776.714338,46.531825
39,6.295292,59.498401,1.042686,241.616781,26.833420
96,6.444180,97.328045,2.424691,464.918301,40.023571


In [17]:
# Method 1: Min-Max Scaling (Normalization)
scaler_minmax = MinMaxScaler()
housing_minmax = pd.DataFrame(
    scaler_minmax.fit_transform(housing_data),
    columns=housing_data.columns
)
print(f"\nAfter Min-Max Scaling (0-1 range):")
print(housing_minmax.describe())


After Min-Max Scaling (0-1 range):
            rooms         age    distance    tax_rate       price
count  100.000000  100.000000  100.000000  100.000000  100.000000
mean     0.562586    0.490669    0.227751    0.493511    0.510826
std      0.203078    0.293848    0.209092    0.307829    0.232215
min      0.000000    0.000000    0.000000    0.000000    0.000000
25%      0.451438    0.240894    0.071709    0.254679    0.346510
50%      0.557419    0.512268    0.168437    0.492188    0.508775
75%      0.676583    0.703266    0.322659    0.759983    0.657671
max      1.000000    1.000000    1.000000    1.000000    1.000000


In [18]:
housing_minmax.sample(5)

,rooms,age,distance,tax_rate,price
77,0.518946,0.671474,0.081489,0.000000,0.318847
34,0.769739,0.086915,0.453326,0.949942,0.828764
45,0.424841,0.172656,0.148119,0.314533,0.370480
62,0.338417,0.913080,0.372966,0.368677,0.122741
73,0.935681,0.911180,0.217402,0.832548,0.231647


In [19]:
# Method 2: Standardization (Z-score normalization)
scaler_standard = StandardScaler()
housing_standard = pd.DataFrame(
    scaler_standard.fit_transform(housing_data),
    columns=housing_data.columns
)
print(f"\nAfter Standardization (mean=0, std=1):")
print(housing_standard.describe())


After Standardization (mean=0, std=1):
              rooms           age      distance      tax_rate         price
count  1.000000e+02  1.000000e+02  1.000000e+02  1.000000e+02  1.000000e+02
mean  -4.923839e-16  2.053913e-16 -1.665335e-17 -3.685940e-16  2.775558e-16
std    1.005038e+00  1.005038e+00  1.005038e+00  1.005038e+00  1.005038e+00
min   -2.784256e+00 -1.678219e+00 -1.094728e+00 -1.611274e+00 -2.210879e+00
25%   -5.500777e-01 -8.542966e-01 -7.500453e-01 -7.797654e-01 -7.111663e-01
50%   -2.557477e-02  7.387473e-02 -2.851051e-01 -4.319214e-03 -8.875651e-03
75%    5.641760e-01  7.271373e-01  4.561927e-01  8.700123e-01  6.355510e-01
max    2.164774e+00  1.742047e+00  3.711960e+00  1.653648e+00  2.117170e+00


In [20]:
housing_standard.sample(5)

,rooms,age,distance,tax_rate,price
40,0.932158,0.667452,0.622709,-1.311531,2.117170
44,-1.521305,0.554471,-0.361938,-1.410143,-2.210879
92,-0.662014,-1.369138,0.018096,-1.343431,-1.267112
64,1.014117,-0.513347,-0.968873,-0.511602,-0.074657
57,-0.227271,0.240651,-0.292687,-1.056043,-0.550654


In [21]:
# Method 3: Robust Scaling (less sensitive to outliers)
scaler_robust = RobustScaler()
housing_robust = pd.DataFrame(
    scaler_robust.fit_transform(housing_data),
    columns=housing_data.columns
)
print(f"\nAfter Robust Scaling:")
print(housing_robust.describe())


After Robust Scaling:
            rooms         age      distance    tax_rate         price
count  100.000000  100.000000  1.000000e+02  100.000000  1.000000e+02
mean     0.022952   -0.046714  2.363589e-01    0.002618  6.590582e-03
std      0.901983    0.635523  8.332002e-01    0.609196  7.462872e-01
min     -2.475810   -1.107914 -6.711964e-01   -0.974043 -1.635089e+00
25%     -0.470721   -0.586918 -3.854465e-01   -0.470031 -5.214833e-01
50%      0.000000    0.000000 -6.418477e-17    0.000000 -1.322727e-16
75%      0.529279    0.413082  6.145535e-01    0.529969  4.785167e-01
max      1.965754    1.054848  3.313662e+00    1.004964  1.578687e+00


In [22]:
# Method 5: Unit Vector Scaling (L2 normalization)
from sklearn.preprocessing import normalize
housing_unit = pd.DataFrame(
    normalize(housing_data, norm='l2'),
    columns=housing_data.columns
)
print(f"\nAfter Unit Vector Scaling:")
print(housing_unit.describe())


After Unit Vector Scaling:
            rooms         age    distance    tax_rate       price
count  100.000000  100.000000  100.000000  100.000000  100.000000
mean     0.013456    0.114497    0.007642    0.987287    0.058955
std      0.006573    0.084448    0.008432    0.015797    0.032080
min      0.004115    0.002017    0.000099    0.917854    0.009250
25%      0.009039    0.045546    0.002119    0.982276    0.034676
50%      0.011040    0.099504    0.004527    0.993308    0.052500
75%      0.016051    0.162321    0.009635    0.997236    0.080074
max      0.034590    0.380479    0.043679    0.999445    0.171531


In [23]:
housing_unit.head(5)

,rooms,age,distance,tax_rate,price
0,0.016181,0.101534,0.011399,0.992177,0.069881
1,0.008838,0.035074,0.003197,0.998904,0.029542
2,0.033224,0.061318,0.012298,0.994280,0.079957
3,0.030308,0.125935,0.007444,0.986454,0.100374
4,0.022780,0.380479,0.002628,0.917854,0.110697


In [24]:
# Create sample dataset with categorical variables
categorical_data = pd.DataFrame({
    'city': ['Mumbai', 'Delhi', 'Bangalore', 'Chennai', 'Mumbai', 'Delhi', 'Pune', 'Bangalore'],
    'education': ['Graduate', 'Post-Graduate', 'Graduate', 'High School', 'Post-Graduate', 'Graduate', 'High School', 'Graduate'],
    'experience_level': ['Junior', 'Senior', 'Mid', 'Junior', 'Senior', 'Mid', 'Junior', 'Senior'],
    'salary_range': ['Low', 'High', 'Medium', 'Low', 'High', 'Medium', 'Low', 'High'],
    'performance': [85, 92, 78, 65, 95, 82, 70, 88]
})

print("Original categorical dataset:")
categorical_data.head(5)

Original categorical dataset:


,city,education,experience_level,salary_range,performance
0,Mumbai,Graduate,Junior,Low,85
1,Delhi,Post-Graduate,Senior,High,92
2,Bangalore,Graduate,Mid,Medium,78
3,Chennai,High School,Junior,Low,65
4,Mumbai,Post-Graduate,Senior,High,95


In [25]:
# Method 1: Label Encoding (for ordinal data)
le_education = LabelEncoder()
le_experience = LabelEncoder()
le_salary = LabelEncoder()

categorical_label = categorical_data.copy()
categorical_label['education_encoded'] = le_education.fit_transform(categorical_label['education'])
categorical_label['experience_encoded'] = le_experience.fit_transform(categorical_label['experience_level'])
categorical_label['salary_encoded'] = le_salary.fit_transform(categorical_label['salary_range'])

print(f"\nAfter Label Encoding:")
print(categorical_label[['education', 'education_encoded', 'experience_level', 'experience_encoded']])


After Label Encoding:
       education  education_encoded experience_level  experience_encoded
0       Graduate                  0           Junior                   0
1  Post-Graduate                  2           Senior                   2
2       Graduate                  0              Mid                   1
3    High School                  1           Junior                   0
4  Post-Graduate                  2           Senior                   2
5       Graduate                  0              Mid                   1
6    High School                  1           Junior                   0
7       Graduate                  0           Senior                   2


In [26]:
categorical_label

,city,education,experience_level,salary_range,performance,education_encoded,experience_encoded,salary_encoded
0,Mumbai,Graduate,Junior,Low,85,0,0,1
1,Delhi,Post-Graduate,Senior,High,92,2,2,0
2,Bangalore,Graduate,Mid,Medium,78,0,1,2
3,Chennai,High School,Junior,Low,65,1,0,1
4,Mumbai,Post-Graduate,Senior,High,95,2,2,0
5,Delhi,Graduate,Mid,Medium,82,0,1,2
6,Pune,High School,Junior,Low,70,1,0,1
7,Bangalore,Graduate,Senior,High,88,0,2,0


In [27]:
# Method 2: One-Hot Encoding (for nominal data)
categorical_onehot = pd.get_dummies(categorical_data, columns=['city', 'education'], prefix=['city', 'edu'])
print(f"\nAfter One-Hot Encoding:")
print(categorical_onehot.head())


After One-Hot Encoding:
  experience_level salary_range  performance  city_Bangalore  city_Chennai  \
0           Junior          Low           85           False         False   
1           Senior         High           92           False         False   
2              Mid       Medium           78            True         False   
3           Junior          Low           65           False          True   
4           Senior         High           95           False         False   

   city_Delhi  city_Mumbai  city_Pune  edu_Graduate  edu_High School  \
0       False         True      False          True            False   
1        True        False      False         False            False   
2       False        False      False          True            False   
3       False        False      False         False             True   
4       False         True      False         False            False   

   edu_Post-Graduate  
0              False  
1               True  
2   

In [28]:
categorical_onehot.head()

,experience_level,salary_range,performance,city_Bangalore,city_Chennai,city_Delhi,city_Mumbai,city_Pune,edu_Graduate,edu_High School,edu_Post-Graduate
0,Junior,Low,85,False,False,False,True,False,True,False,False
1,Senior,High,92,False,False,True,False,False,False,False,True
2,Mid,Medium,78,True,False,False,False,False,True,False,False
3,Junior,Low,65,False,True,False,False,False,False,True,False
4,Senior,High,95,False,False,False,True,False,False,False,True


In [29]:
# Method 5: Ordinal Encoding (when order matters)
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[['Low', 'Medium', 'High']])
salary_ordinal = ordinal_encoder.fit_transform(categorical_data[['salary_range']])
print(f"\nOrdinal Encoding for 'salary_range':")
print(f"Original: {categorical_data['salary_range'].tolist()}")
print(f"Encoded: {salary_ordinal.flatten().tolist()}")


Ordinal Encoding for 'salary_range':
Original: ['Low', 'High', 'Medium', 'Low', 'High', 'Medium', 'Low', 'High']
Encoded: [0.0, 2.0, 1.0, 0.0, 2.0, 1.0, 0.0, 2.0]


In [30]:
categorical_data['salary_range'] = ordinal_encoder.fit_transform(categorical_data[['salary_range']])

In [31]:
categorical_data

,city,education,experience_level,salary_range,performance
0,Mumbai,Graduate,Junior,0.0,85
1,Delhi,Post-Graduate,Senior,2.0,92
2,Bangalore,Graduate,Mid,1.0,78
3,Chennai,High School,Junior,0.0,65
4,Mumbai,Post-Graduate,Senior,2.0,95
5,Delhi,Graduate,Mid,1.0,82
6,Pune,High School,Junior,0.0,70
7,Bangalore,Graduate,Senior,2.0,88


In [32]:
# Create sample dataset for binning
age_data = pd.DataFrame({
    'age': [22, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 18, 28, 33, 38, 42, 48, 52, 58, 63],
    'income': [30000, 45000, 60000, 75000, 80000, 90000, 95000, 100000, 85000, 70000,
               60000, 25000, 50000, 70000, 78000, 85000, 92000, 98000, 88000, 75000]
})

In [33]:
age_data

,age,income
0,22,30000
1,25,45000
2,30,60000
3,35,75000
4,40,80000
5,45,90000
6,50,95000
7,55,100000
8,60,85000
9,65,70000


In [34]:
# Method 1: Equal-width binning
age_data['age_bins_equal'] = pd.cut(age_data['age'], bins=4, labels=['Young', 'Adult', 'Middle-aged', 'Senior'])
print(f"\nEqual-width binning for age:")
print(age_data[['age', 'age_bins_equal']].head(10))


Equal-width binning for age:
   age age_bins_equal
0   22          Young
1   25          Young
2   30          Young
3   35          Adult
4   40          Adult
5   45    Middle-aged
6   50    Middle-aged
7   55    Middle-aged
8   60         Senior
9   65         Senior


In [35]:
age_data.head(5)

,age,income,age_bins_equal
0,22,30000,Young
1,25,45000,Young
2,30,60000,Young
3,35,75000,Adult
4,40,80000,Adult


In [36]:
# Method 2: Equal-frequency binning (quantile-based)
age_data['age_bins_quantile'] = pd.qcut(age_data['age'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
print(f"\nEqual-frequency binning for age:")
print(age_data[['age', 'age_bins_quantile']].head(10))


Equal-frequency binning for age:
   age age_bins_quantile
0   22                Q1
1   25                Q1
2   30                Q1
3   35                Q2
4   40                Q2
5   45                Q3
6   50                Q3
7   55                Q3
8   60                Q4
9   65                Q4


In [37]:
age_data.head(5)

,age,income,age_bins_equal,age_bins_quantile
0,22,30000,Young,Q1
1,25,45000,Young,Q1
2,30,60000,Young,Q1
3,35,75000,Adult,Q2
4,40,80000,Adult,Q2


In [38]:
custom_bins = [0, 30, 50, 70, 100]
custom_labels = ['Youth', 'Young Adult', 'Middle Age', 'Senior']
age_data['age_bins_custom'] = pd.cut(age_data['age'], bins=custom_bins, labels=custom_labels)
print(f"\nCustom binning for age:")
print(age_data[['age', 'age_bins_custom']].head(10))


Custom binning for age:
   age age_bins_custom
0   22           Youth
1   25           Youth
2   30           Youth
3   35     Young Adult
4   40     Young Adult
5   45     Young Adult
6   50     Young Adult
7   55      Middle Age
8   60      Middle Age
9   65      Middle Age


In [39]:
age_data.head(5)

,age,income,age_bins_equal,age_bins_quantile,age_bins_custom
0,22,30000,Young,Q1,Youth
1,25,45000,Young,Q1,Youth
2,30,60000,Young,Q1,Youth
3,35,75000,Adult,Q2,Young Adult
4,40,80000,Adult,Q2,Young Adult
